Connected to PRL (Python 3.11.0)

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
import pandas as pd
from tqdm import tqdm
import random
from utils import RuleEvaluation, DDQNEvaluation, Plotter
from agent import DDQNAgent, TemporalDDQNAgent
from TestEnv import Electric_Car
seed = 2705
TRAIN = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load Data

def elongate(df):
    df_long = pd.wide_to_long(df, i = "PRICES", j = "hour", stubnames=["Hour"], sep = " ").reset_index()
    df_long.rename(columns={"Hour": "price", "PRICES": "date"}, inplace = True)
    df_long['datetime'] = pd.to_datetime(df_long['date']) + pd.to_timedelta(df_long['hour'], unit='h')
    df_long.sort_values(['datetime'], ascending=[True], inplace=True)
    df_long['price'] = df_long['price'].astype(float) 
    return df_long.reset_index(drop=True)

train_name = 'data/train.xlsx'
val_name = 'data/validate.xlsx'
train = elongate(pd.read_excel(train_name))
val = elongate(pd.read_excel(val_name))
features_train = pd.read_csv('data/features_train.csv')
features_val = pd.read_csv('data/features_val.csv')
#%%
# Define the intervals for gamma and reward shaping factor

gamma_interval = [0.93, 0.99]
reward_shaping_interval = [0.2, 0.5]
battery_factor_interval = [0.0, 0.15]
# Define the number of iterations for the random search

num_iterations = 100
for i in range(num_iterations):
    # Generate random values within the intervals
    gamma = np.random.uniform(*gamma_interval)
    reward_shaping_factor = np.random.uniform(*reward_shaping_interval)
    battery_factor = np.random.uniform(*battery_factor_interval)
    print(f'\nIteration: {i+1}, Gamma: {gamma}, Reward Shaping Factor: {reward_shaping_factor}, Battery Factor: {battery_factor}')
    seed = 2705
    rep = 420000
    batch_size = 48
    gamma = gamma
    epsilon = 1.0
    epsilon_decay = 99999
    epsilon_min = 0.1
    learning_rate = 5e-5
    price_horizon = 48
    future_horizon = 0
    hidden_dim = 128
    num_layers = 4
    positions = False
    action_classes = 3
    reward_shaping = True
    factor = reward_shaping_factor
    verbose = False
    normalize = True
    df = train_name
    # Initialize Environment
    env = Electric_Car(path_to_test_data=df)
    val_env = Electric_Car(path_to_test_data=val_name)
    #Initialize DQN
    agent = DDQNAgent(env = env,
                    features = features_train,
                    epsilon_decay = epsilon_decay,
                    epsilon_start = epsilon,
                    epsilon_end = epsilon_min,
                    discount_rate = gamma,
                    lr = learning_rate,
                    buffer_size = 100000,
                    price_horizon = price_horizon,
                    hidden_dim=hidden_dim,
                    num_layers = num_layers,
                    positions = positions,
                    action_classes = action_classes, 
                    reward_shaping = reward_shaping,
                    shaping_factor = factor,
                    normalize = normalize,
                    verbose = verbose)
    val_agent = DDQNAgent(env = val_env,
                    features = features_val,
                    epsilon_decay = epsilon_decay,
                    epsilon_start = epsilon,
                    epsilon_end = epsilon_min,
                    discount_rate = gamma,
                    lr = learning_rate,
                    buffer_size = 100000,
                    price_horizon = price_horizon,
                    hidden_dim=hidden_dim,
                    num_layers = num_layers,
                    positions = positions,
                    action_classes = action_classes, 
                    reward_shaping = reward_shaping,
                    shaping_factor = factor,
                    normalize = normalize,
                    verbose = verbose)
    episode_balance = 0
    episode_loss = 0
    episode_counter = 0
    episode_reward = 0
    obs, r, t, _, _ = env.step(random.randint(-1,1)) # Reset environment and get initial observation
    state, grads = agent.obs_to_state(obs)
    for i in tqdm(range(rep)):
        action = agent.choose_action(i, state, greedy = False) # Choose action (discrete)
        cont_action = agent.action_to_cont(action) # Convert to continuous action
        new_obs, r, t, _, _ = env.step(cont_action)
        new_state, new_grads = agent.obs_to_state(new_obs)
        # Reward Shaping            
        new_reward = agent.shape_reward(r, cont_action, grads, battery_factor = battery_factor)
        # Fill replay buffer - THIS IS THE ONLY THING WE DO WITH THE CURRENT OBSERVATION - LEARNING IS FULLY PERFORMED FROM THE REPLAY BUFFER
        if state.shape[0] == agent.state_dim and new_state.shape[0] == agent.state_dim:
            agent.replay_memory.add_data((state, action, new_reward, t, new_state))
        #Update DQN
        loss = agent.optimize(batch_size)
        # Update values
        episode_balance += r
        episode_reward += r
        episode_loss += loss
        # New observation
        state = new_state
        grads = new_grads # Gradients for reward shaping
        if t:
            # Reset Environment
            env.counter = 0
            env.hour = 1
            env.day = 1
            episode_counter += 1
            if episode_counter % 4 == 0:
                print('Episode ', episode_counter, 'Balance: ', episode_balance, 'Reward: ', episode_reward, 'Loss: ', episode_loss) # Add both balance and reward to see how training objective and actually spent money differ
            # Scheduler Step
            agent.scheduler.step(episode_loss)
            episode_loss = 0
            episode_balance = 0
            episode_reward = 0
            if episode_counter % 4 == 0:
                # Evaluate DQN
                print("Training Evaluation")
                train_dqn = DDQNEvaluation(price_horizon = price_horizon)
                train_dqn.evaluate(agent = agent)
                # Evaluate DQN
                print("Validation Evaluation")
                val_agent.dqn_predict.load_state_dict(agent.dqn_predict.state_dict())
                val_dqn = DDQNEvaluation(price_horizon = price_horizon)
                val_dqn.evaluate(agent = val_agent)
                # Reset Environment
                env.counter = 0
                env.hour = 1
                env.day = 1
                val_env.counter = 0
                val_env.hour = 1
                val_env.day = 1
    # Save agent
    torch.save(agent.dqn_predict.state_dict(), f'models/random_search_gamma{gamma}_factor_{factor}.pt')


Iteration: 1, Gamma: 0.9386139862524565, Reward Shaping Factor: 0.41420794369762703, Battery Factor: 0.06555965628369004


 25%|██▍       | 104155/420000 [06:31<21:58, 239.61it/s]

Episode  4 Balance:  -1886.3773093209911 Reward:  -1886.3773093209911 Loss:  553.1817215082701
Training Evaluation
Absolute Balance:  -164.01173197530866
Validation Evaluation


 25%|██▍       | 104191/420000 [06:56<19:59:09,  4.39it/s]

Absolute Balance:  -477.6077298765432


...

 75%|███████▍  | 314567/420000 [29:32<07:11, 244.19it/s]  

Episode  12 Balance:  -857.4889871604922 Reward:  -857.4889871604922 Loss:  496.89044427662157
Training Evaluation
Absolute Balance:  17.69481475308636
Validation Evaluation


 75%|███████▍  | 314608/420000 [29:56<6:40:44,  4.38it/s]

Absolute Balance:  -433.87253802469144


 81%|████████  | 340850/420000 [32:59<09:48, 134.52it/s] 